# In this notebook I'm going to learn some of the most fundamental concepts of tensors using TensorFlow

More spefically, I'm going to learn:
* Intruduction to tensors
* Getting information from tensors
* Manipulatings tensors
* Tensors & NumPy
* Using @tf.function (a way to speed up regular Python functions)
* Using GPUs with TensorFlow (or TPUs)
* Exercise to learn even more!

### Introduction to tensors

In [1]:
# Import TensorFlow
import tensorflow as tf
import numpy as np
print(tf.__version__)

2.10.0


In [2]:
# Creating tensors with tf.constant()
scalar = tf.constant(7)
scalar

<tf.Tensor: shape=(), dtype=int32, numpy=7>

In [3]:
# Check the number of dimension of a tensor (ndim stands for number of dimensions)
scalar.ndim

0

In [4]:
# Create a vector
vector = tf.constant([10, 10])
vector

<tf.Tensor: shape=(2,), dtype=int32, numpy=array([10, 10], dtype=int32)>

In [5]:
# Check the dimensions of vector
vector.ndim

1

In [6]:
# Create a matrix (more than one dimension)
matrix = tf.constant([[10,7], 
                      [7,10]])
matrix

<tf.Tensor: shape=(2, 2), dtype=int32, numpy=
array([[10,  7],
       [ 7, 10]], dtype=int32)>

In [7]:
# Check the dimensions of the matrix
matrix.ndim

2

In [8]:
# Create another matrix
another_matrix = tf.constant([[10., 7],
                              [3., 2.],
                              [8., 9.]], dtype=tf.float16) # specify the datatype with dtype

another_matrix

<tf.Tensor: shape=(3, 2), dtype=float16, numpy=
array([[10.,  7.],
       [ 3.,  2.],
       [ 8.,  9.]], dtype=float16)>

In [9]:
# What's the number of dimensions of another_matrix?
another_matrix.ndim

2

In [10]:
# Let's create a tensor
tensor = tf.constant([[[1, 2, 3],
                       [4, 5, 6]],
                      [[7, 8, 9],
                       [10, 11, 12]],
                      [[13, 14, 15],
                       [16, 17, 18]]])
tensor

<tf.Tensor: shape=(3, 2, 3), dtype=int32, numpy=
array([[[ 1,  2,  3],
        [ 4,  5,  6]],

       [[ 7,  8,  9],
        [10, 11, 12]],

       [[13, 14, 15],
        [16, 17, 18]]], dtype=int32)>

In [11]:
tensor.ndim

3

What we've created so far:
* scalar - single number
* vector - number with direction (e.g. wind speed and direction)
* matrix - 2-dimensional array of numbers
* tensor - n-dimensional array of numbers (n can be any number)

### Creating tensors with `tf.Variable`

In [12]:
tf.Variable

tensorflow.python.ops.variables.Variable

In [13]:
# Create the same tensor with tf.Variable() as above
changeable_tensor = tf.Variable([10, 7])
unchangeable_tensor = tf.constant([10, 7])
changeable_tensor, unchangeable_tensor

(<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([10,  7], dtype=int32)>,
 <tf.Tensor: shape=(2,), dtype=int32, numpy=array([10,  7], dtype=int32)>)

In [14]:
# Changing one of the elements in changeable_tensor
changeable_tensor[0] = 7

TypeError: ignored

In [18]:
# How about using .assign()?
changeable_tensor[0].assign(7)
changeable_tensor

<tf.Variable 'Variable:0' shape=(2,) dtype=int32, numpy=array([7, 7], dtype=int32)>

In [ ]:
# Trying to change on unchangeable_tensor
unchangeable_tensor[0].assign(7)
unchangeable_tensor

### Creating random tensors
Random tensors are tensors of some arbitrary size which contain random numbers.

In [ ]:
# Creating two random (but the same) tensors
random_1 = tf.random.Generator.from_seed(7) # set seed for reproducibility
random_1 = random_1.normal(shape=(3,2))
random_2 = tf.random.Generator.from_seed(7)
random_2 = random_2.normal(shape=(3,2))

# Are the equal?
random_1, random_2, random_1 == random_2

### Shuffling the order of elements in a tensor

In [ ]:
# Shuffle a tensor (when you want to shuffle your data so order does not affect learning)
not_shuffled = tf.constant([[10, 7],
                            [3, 4],
                            [2, 5]])
not_shuffled.ndim

In [ ]:
# Shuffle not_shuffled tensor
tf.random.shuffle(not_shuffled)

In [ ]:
# Shuffle our not_shuffled tensor
tf.random.set_seed(42) # global level random seed
tf.random.shuffle(not_shuffled, seed=42) # operation level random seed

It looks like if we want our shuffled tensors to be in the same order, we've got to use the global level random seed as well as the operational level random seed:

> Rule 4: "If both the global and the operation seed are set: Both seeds are used in conjuncion to determine to random sequence."

### Other ways to create tensors

In [ ]:
# Create a tensor of ones
tf.ones([10, 7])

In [ ]:
# Create a tensor of zeros
tf.zeros([7, 10])

### Turn NumPy arrays into tensors
The main difference between NumPy arrays and TensorFlow tensors is that tensors can be run on a GPU (much faster for numerical computing).

In [ ]:
# We can also turn NumPy arrays into tensors
import numpy as np
numpy_A = np.arange(1, 25, dtype=np.int32) # create a NumPy array between 1 and 25
numpy_A

# Convention as follows:
# X = tf.constant(some_matrix) # capital for matrix or tensor
# y = tf.constant(vector) # non-capital for vector

In [ ]:
A = tf.constant(numpy_A, shape=(3, 8)) # tensor
B = tf.constant(numpy_A) # vector
A, B

### Getting information from tensors
* Shape (number of elements of each of the dimensions of a tensor)
* Rank (number of dimensions, e.g. scalar, vector, matrix, tensor)
* Axis or dimension (a particular dimension of a tensor)
* Size (total number of items in tensor)

In [ ]:
# Create a rank 4 tensor (4 dimensions)
rank_4_tensor = tf.zeros(shape=[2,3,4,5])
rank_4_tensor

In [ ]:
rank_4_tensor[0]

In [ ]:
rank_4_tensor.shape, rank_4_tensor.ndim, tf.size(rank_4_tensor)

In [ ]:
# size is multiplying each dimension
2 * 3 * 4 * 5

In [ ]:
# Getting various attributes of our tensor
print('Datatype of every element:', rank_4_tensor.dtype)
print('Number of dimensions (rank):', rank_4_tensor.ndim)
print('Shape of tensor:', rank_4_tensor.shape)
print('Elements along the 0 axis:', rank_4_tensor.shape[0])
print('Elements along the last axis:', rank_4_tensor.shape[-1])
print('Total number of elements in tensor:', tf.size(rank_4_tensor))
print('Total number of elements in tensor:', tf.size(rank_4_tensor).numpy())

### Indexing and expanding tensors

Tensors can be indexed just like Python lists.

In [ ]:
# Get the first two elements of each dimension
rank_4_tensor[:2, :2, :2, :2]

In [ ]:
# Get the first element from the each dimension from each index except for the final one
rank_4_tensor[:1, :1, :1]

In [ ]:
rank_4_tensor[:1, :1, :, :1]

In [ ]:
rank_4_tensor[:1, :, :1, :1]

In [ ]:
# Create a rank 2 tensor (2 dimensions)
rank_2_tensor = tf.constant([[10, 7],
                             [3, 4]])
rank_2_tensor.shape, rank_2_tensor.ndim

In [ ]:
# Get the last item of each row of our rank 2 tensor
rank_2_tensor[:, -1]

In [ ]:
# Adding an extra dimensions to rank 2 tensor
rank_3_tensor = rank_2_tensor[..., tf.newaxis]
rank_3_tensor

In [ ]:
# Alternative to tf.newaxis
tf.expand_dims(rank_2_tensor, axis=-1) # '-1' means expand the final axis

In [ ]:
# Expand the 0-axis
tf.expand_dims(rank_2_tensor, axis=0) # exapand the 0-axis

### Manipulating tensors (tensor operations)
**Basic operations**<br>
`+`, `-`, `*`, `/`

In [ ]:
# You can add values to a tensor using the addition operator
tensor = tf.constant([[10, 7], 
                      [3, 4]])
tensor + 10

In [ ]:
# Original tensor is unchanged
tensor

In [ ]:
# If we want the change - overwrite the value
tensor = tensor + 10
tensor

In [ ]:
# Multiplication
tensor * 10

In [ ]:
# Substraction
tensor - 10

In [ ]:
# We can use the TensorFlow built-in function too
# The main advantage is to calculate on GPU as we use tensors
tf.multiply(tensor, 10)

**Matrix multiplication**<br>
In machine learning, matrix multiplication is one of the most common tensor operations.

There are two rules our tensors (or matricies) need to fulfill if we're going to matrix multiply them:

1. The inner dimensions must match
2. The resulting matrix has the shape of the outer dimensions

In [ ]:
# Matrix multiplication in TensorFlow
print(tensor)

In [ ]:
tf.matmul(tensor, tensor)

In [ ]:
# Exercise from the video
tensor1 = tf.constant([[1,2,5], 
                       [7,2,1], 
                       [3,3,3]])

tensor2 = tf.constant([[3,5], 
                        [6,7], 
                        [1,8]])

tf.matmul(tensor1, tensor2)

In [ ]:
# Matrix multiplication with Python operator "@"
tensor @ tensor

In [ ]:
tensor.shape

In [ ]:
# Create a tensor or (3, 2)
X = tf.constant([[1,2], 
                 [3,4], 
                 [5,6]])

# Create another (3, 2) tensor
Y = tf.constant([[7,8], 
                 [9, 10], 
                 [11, 12]])

X, Y

In [ ]:
# Let's try to matrix multiply tensor of the same shape
X @ Y

In [ ]:
tf.matmul(X, Y)

In [ ]:
# Reshaping the 2nd matrix to fit 
Y = tf.reshape(Y, shape=(2,3))
X, Y

In [ ]:
X.shape, tf.reshape(Y, shape=(2,3)).shape

In [ ]:
# Python matrix multiplication
X @ tf.reshape(Y, shape=(2,3))

In [ ]:
# Try to matrix multiply X by reshaped Y
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

In [ ]:
tf.reshape(X, shape=(2, 3)).shape, Y.shape

In [ ]:
# Reversing the order of the reshape, so reshaping X tensor
tf.matmul(tf.reshape(X, shape=(2, 3)), Y)

In [ ]:
# Can do the same with transpose
X, tf.transpose(X), tf.reshape(X, shape=(2, 3))

In [ ]:
# Try matrix multiplication with transpose
tf.matmul(tf.transpose(X), Y)

**The dot product**

Matrix multiplication is also referred as dot product.

You can perform matrix multiplication using:
* `tf.matmul()`
* `tf.tensordot()`
* `@`


In [ ]:
X, Y

In [ ]:
# Perform the dot product on X and Y (requires X or Y to be transposed)
tf.tensordot(tf.transpose(X), Y, axes=1)

In [ ]:
# Perfrom matrix multiplication between X and Y (transposed)
tf.matmul(X, tf.transpose(Y))

In [ ]:
# Perform matrix multipication between X and Y (reshaped)
tf.matmul(X, tf.reshape(Y, shape=(2, 3)))

In [ ]:
# Check the values of Y, reshape Y and transposed Y
print('Regular Y:')
print(Y, '\n')

print('Y reshaped to (2, 3) shape:')
print(tf.reshape(Y, shape=(2, 3)), "\n")

print('Y transposed:')
print(tf.transpose(Y))



Generally, when performing matrix multiplication on two tensors and one of the axex doesn't line up, you will transpose (rather than reshape) one of the tensors to satisfy matrix multiplication rules.

### Change the datatype of the tensor

In [ ]:
# Create a new tensor with default data type (float32)
B = tf.constant([1.7, 7.4])
B.dtype

In [ ]:
C = tf.constant([1,7])
C.dtype

In [ ]:
# Change from float32 to float16 - reduced precision
D = tf.cast(B, dtype=tf.float16)
D.dtype

In [ ]:
# Change from int32 to float32
E = tf.cast(C, dtype=tf.float32)
E.dtype

### Aggregating tensors
Aggregating tensors = condensing them from multiple values down to a smaller amount of values.

In [ ]:
# Getting the absolute values
D = tf.constant([-7, -10])
tf.abs(D)

Let's go through the following forms of aggregation:
* get the minimum
* get the maximum
* get the mean of the tensor
* get the sum of the tensor

In [ ]:
# Create a random tensor with values between 0 and 100 and size of 50
E = tf.constant(np.random.randint(0,100, size=50))
E

In [ ]:
tf.size(E), E.shape, E.ndim

In [ ]:
# Getting the max value
tf.reduce_max(E)

In [ ]:
# Getting the min value
tf.reduce_min(E)

In [ ]:
# Getting the mean
tf.reduce_mean(E)

In [ ]:
# Getting the sum
tf.reduce_sum(E)

In [ ]:
# Find the variance and standard deviation
E = tf.cast(E, dtype=tf.float16)
tf.math.reduce_variance(E), tf.math.reduce_std(E)

### Find the positional maximum and minimum


In [ ]:
# Create a new tensor for findings
tf.random.set_seed(42)
F = tf.random.uniform(shape=[50])
F

In [ ]:
# Positional minimum
tf.argmin(F)

In [ ]:
# Find the minimum using the poitional minimum index
F[tf.argmin(F)]

In [ ]:
# Positional maximum
tf.argmax(F)

In [ ]:
# Index on largest value position
F[tf.argmax(F)]

In [ ]:
# Find the max value of F
tf.reduce_max(F)

In [ ]:
# Check for equality
F[tf.argmax(F)] == tf.reduce_max(F)

### Squeezing a tensor (removing all single dimensions)

In [ ]:
# Create a tensor
tf.random.set_seed(42)
G = tf.constant(tf.random.uniform(shape=[50]), shape=(1,1,1,1,50))
G

In [ ]:
G.shape

In [ ]:
G_squezzed = tf.squeeze(G)
G_squezzed, G_squezzed.shape

### One-hot encoding tensors

In [ ]:
# Create a list of indicies
some_list = [0, 1, 2, 3] # could be red, green, blue purple

# One-hot encode our list of indicies
tf.one_hot(some_list, depth=4)

In [ ]:
# Specify custom values
tf.one_hot(some_list, depth=4, 
           on_value='some on value', 
           off_value='different off value')

### Squaring, log, square root

In [ ]:
# Create a new tensor
H = tf.range(1,10)
H

In [ ]:
# Square it
tf.square(H)

In [ ]:
# Square root (will error, method requires non-integer type)
tf.sqrt(H)

In [ ]:
# Square root
tf.sqrt(tf.cast(H, dtype=tf.float32))

In [ ]:
# Find the log
tf.math.log(tf.cast(H,dtype=tf.float32))

### Tensors any NumPy

TensorFlow interacts beautifully with NumPy arrays.

In [ ]:
# Create a tensorm directly from a NumPy array
J = tf.constant(np.array([10., 7., 4.]))
J

In [ ]:
# Convert tenosr back to NumPy array
np.array(J), type(np.array(J))

In [ ]:
# Convert tensor J into NumPy array
J.numpy(), type(J.numpy())

In [ ]:
# The default types of each are slightly different
numpy_J = tf.constant(np.array([10., 7., 4.]))
tensor_J = tf.constant([10., 7., 4.])

# Check the datatypes of each
numpy_J.dtype, tensor_J.dtype

### Finding access to GPUs

In [15]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [16]:
tf.config.list_physical_devices("GPU")

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [17]:
!nvidia-smi

Sat Dec 17 11:43:15 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P0    29W /  70W |    314MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------